In [5]:
# TODO Please update the project id before running the code



In [21]:
%%writefile ../pipeline/pipeline_source.py
#This is a simple pipeline that is used to load and run pipeline

import kfp
from kfp.v2 import compiler
from kfp.v2.google.client import AIPlatformClient
from google.cloud import aiplatform
from google.cloud import aiplatform as aip
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2 import compiler

# Change the Project ID
#PROJECT_ID ="[YOUR PROJECT ID]"
#BUCKET_LOC = "[Your staging bucket locations]"
#PIPELINE_NAME="[Your Pipeline Name]"

PROJECT_ID ="demogct"
BUCKET_LOC = "gs://demogct/vipipelines/"
PIPELINE_NAME="sd-vertex-pipeline"


aip.init(project=PROJECT_ID, staging_bucket=BUCKET_LOC)

@kfp.dsl.pipeline(name=PIPELINE_NAME)
def pipeline(project: str = PROJECT_ID):
    ds_op = gcc_aip.ImageDatasetCreateOp(
        project=project,
        display_name="flowers",
        gcs_source="gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv",
        import_schema_uri=aip.schema.dataset.ioformat.image.single_label_classification,
    )

    training_job_run_op = gcc_aip.AutoMLImageTrainingJobRunOp(
        project=project,
        display_name="train-automl-flowers",
        prediction_type="classification",
        model_type="CLOUD",
        base_model=None,
        dataset=ds_op.outputs["dataset"],
        model_display_name="train-automl-flowers",
        training_fraction_split=0.7,
        validation_fraction_split=0.2,
        test_fraction_split=0.1,
        budget_milli_node_hours=8000,
    )
    # 0.1.7 is needed currently to address a bug in the latest container image
    gcc_aip.ModelDeployOp.component_spec.implementation.container.image = ("gcr.io/ml-pipeline/google-cloud-pipeline-components:0.1.7")
 
    
    endpoint_op = gcc_aip.ModelDeployOp(  
         model=training_job_run_op.outputs["model"],
         project="demogct"
    )
  


compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="artifacts/image classification_pipeline.json".replace(" ", "_"),
)

Overwriting ../pipeline/pipeline_source.py


In [23]:
%%writefile ../pipeline/pipeline_execute.py

import kfp
from kfp.v2 import compiler
from kfp.v2.google.client import AIPlatformClient
from google.cloud import aiplatform
from google.cloud import aiplatform as aip
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2 import compiler
from datetime import datetime

# Change the Project ID
#PROJECT_ID ="[YOUR PROJECT ID]"
#BUCKET_LOC = "[Your staging bucket locations]"
#PIPELINE_NAME="[Your Pipeline Name]"

PROJECT_ID ="demogct"
BUCKET_LOC = "gs://demogct/vipipelines/"
PIPELINE_NAME="sd-vertex-pipeline"

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
DISPLAY_NAME = "flowers_" + TIMESTAMP
PIPELINE_ROOT="gs://demogct/vipipelines/"

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="artifacts/image classification_pipeline.json".replace(" ", "_"),
    pipeline_root=PIPELINE_ROOT,
)

job.run()

Overwriting ../pipeline/pipeline_execute.py


In [16]:
import os

TRAIN_VERSION = "tf-gpu.2-6"
DEPLOY_VERSION = "tf2-gpu.2-6"
CONTAINER_ARTIFACTS_DIR="../artifacts"

TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(DEPLOY_VERSION)

dockerfile = f'''FROM {TRAIN_IMAGE}

WORKDIR /root

RUN pip install google-cloud-pipeline-components==0.1.7 google-cloud-aiplatform google-cloud-storage

# Copies the trainer code to the docker image.
COPY ../pipelines/pipeline.py /root/pipeline.py

ENTRYPOINT ["python3", "pipeline.py"]
'''

with open(os.path.join(CONTAINER_ARTIFACTS_DIR, 'Dockerfile'), 'w') as f:
    f.write(dockerfile)

In [17]:
REPOSITORY="day4-workshop"
INITIALS ="mayo"

In [18]:
!gcloud artifacts repositories create $REPOSITORY --repository-format=docker \
--location=us-central1 --description=f"{INITIALS} Vertex sample repository"

Create request issued for: [day4-workshop]
Waiting for operation [projects/demogct/locations/us-central1/operations/044bae
67-4950-44ba-9eb4-b10436898f74] to complete...done.                            
Created repository [day4-workshop].


In [19]:
!gcloud auth configure-docker us-central1-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.
